In [1]:
import itertools

# Our numerical workhorses
import numpy as np
import pandas as pd
import scipy.integrate

# Import Altair for high level plotting
import altair as alt
import altair_catplot as altcat

# Import Bokeh modules for interactive plotting
import bokeh.io
import bokeh.plotting

# Set up Bokeh for inline viewing
bokeh.io.output_notebook()

# Pevent bulky altair plots
alt.data_transformers.enable('json')

Loading BokehJS ...

DataTransformerRegistry.enable('json')

In [2]:
# Load in the genotype file, call it df_gt for genotype DataFrame
df_gt = pd.read_csv("../data/fish_activity_for_validation/150717_2A_genotype_3.txt", 
                  delimiter='\t', comment ='#')

In [3]:
# Tidy the DataFrame
df_gt = pd.melt(df_gt, var_name='genotype', value_name='location')

# Drop all rows that have a NaN in them
df_gt = df_gt.dropna()

df_gt = df_gt.reset_index(drop=True)

df_gt.loc[:,'location'] = df_gt.loc[:, 'location'].astype(int)

In [4]:
df_gt.head(10)

,genotype,location
0,wt,1
1,wt,3
2,wt,8
3,wt,12
4,wt,21
5,wt,28
6,wt,31
7,wt,33
8,wt,34
9,wt,35


In [5]:
# Now, load in the CSV file
fname = '../data/fish_activity_for_validation/150717_2A_2B.csv'
df = pd.read_csv(fname, comment='#')

First, we want to test that all the columns are what we excpect them to be before we start tidying the data.

In [6]:
def test_column_names(df, fname):
    """Ensure DataFrame has proper columns."""
    column_names = ['location','animal','user','sn','an','datatype','start','end','startreason',
                    'endreason','frect', 'fredur', 'midct', 'middur', 'burct', 'burdur', 'stdate',
                    'sttime']

    assert list(df.columns) == column_names, fname + ' has wrong column names.'

In [7]:
test_column_names(df, fname)

Now, we get rid of the c from location column, and convert to an integer type so we can merge this dataframe with the genotype dataframe.

In [8]:
sanititize_loc = lambda x: int(x.replace("c", ""))

df['location'] = df['location'].apply(sanititize_loc)

In [9]:
# We perform the merge.
df = pd.merge(df, df_gt)

We are only looking for fish in instrument 2A, numbered 1 through 96 in the activity data file, so we can get rid of fish in instrument 2B, numbered 97 and above, as they are not used.

In [10]:
df.head()

,location,animal,user,sn,an,datatype,start,end,startreason,endreason,frect,fredur,midct,middur,burct,burdur,stdate,sttime,genotype
0,1,z001,ZEBRALAB02\zebralab_user,1,0,quant,0.0,60.0,Beginning of session,End of period,0,60.0,0,0.0,0,0.0,17/07/2015,14:29:59,wt
1,1,z001,ZEBRALAB02\zebralab_user,1,0,quant,60.0,120.0,End of period,End of period,0,60.0,0,0.0,0,0.0,17/07/2015,14:30:59,wt
2,1,z001,ZEBRALAB02\zebralab_user,1,0,quant,120.0,180.0,End of period,End of period,6,59.0,6,0.9,0,0.0,17/07/2015,14:31:59,wt
3,1,z001,ZEBRALAB02\zebralab_user,1,0,quant,180.0,240.0,End of period,End of period,0,60.0,0,0.0,0,0.0,17/07/2015,14:32:59,wt
4,1,z001,ZEBRALAB02\zebralab_user,1,0,quant,240.0,300.0,End of period,End of period,0,60.0,0,0.0,0,0.0,17/07/2015,14:33:59,wt


In [11]:
df["location"] = df[df["location"] < 97]
df.tail()

,location,animal,user,sn,an,datatype,start,end,startreason,endreason,frect,fredur,midct,middur,burct,burdur,stdate,sttime,genotype
325396,96,z096,ZEBRALAB02\zebralab_user,1,0,quant,246840.0,246900.0,End of period,End of period,4,59.6,4,0.4,0,0.0,20/07/2015,11:03:59,het
325397,96,z096,ZEBRALAB02\zebralab_user,1,0,quant,246900.0,246960.0,End of period,End of period,28,55.2,29,4.8,0,0.0,20/07/2015,11:04:59,het
325398,96,z096,ZEBRALAB02\zebralab_user,1,0,quant,246960.0,247020.0,End of period,End of period,69,50.2,68,9.9,0,0.0,20/07/2015,11:05:59,het
325399,96,z096,ZEBRALAB02\zebralab_user,1,0,quant,247020.0,247080.0,End of period,End of period,62,51.6,63,8.4,0,0.0,20/07/2015,11:06:59,het
325400,96,z096,ZEBRALAB02\zebralab_user,1,0,quant,247080.0,247103.7,End of period,End of session,27,19.3,26,4.4,0,0.0,20/07/2015,11:07:59,het


We want to check that the 96 fish that we *should* have left are present and accounted for.

In [12]:
def check_num_fish(df, num_fish):
    
    fish_array = list(set(df["location"].values))

    if len(fish_array) != num_fish: 
        lost_fish = list(set(range(1, num_fish + 1)) - set(fish_array))
        
        return"Oh no! We've lost {} fishies at indicies: {}. We should have {} but we have {}!" \
                    .format(len(lost_fish), lost_fish, num_fish, num_fish - len(lost_fish))

    return "All fishies are accounted for! Thank the lord!"

In [13]:
check_num_fish(df, 96)

"Oh no! We've lost 17 fishies at indicies: [64, 65, 66, 67, 36, 68, 69, 70, 11, 47, 51, 83, 84, 22, 55, 62, 95]. We should have 96 but we have 79!"

Yikes! Unfortunately we are missed a good chunk of fish! This is good information to know.

We don't want any of our numerical columns to be negative; we'll write a general function for it and apply it to all the numerical columns.

In [47]:
# Loop through all the numerical columns (given in cols) and check that they are positive
def num_negative_col(df, cols):
    results = []
    for col in cols:
       results.append("The column {} has {} negative values".format (col, (df[col] < 0).sum()))
    return results

In [48]:
cols = ["location", "sn", "an", "start", "end", "frect", "fredur", "midct", "burct", "burdur"]
num_negative_col(df, cols)

['The column location has 0 negative values',
 'The column sn has 0 negative values',
 'The column an has 0 negative values',
 'The column start has 0 negative values',
 'The column end has 0 negative values',
 'The column frect has 0 negative values',
 'The column fredur has 0 negative values',
 'The column midct has 0 negative values',
 'The column burct has 0 negative values',
 'The column burdur has 0 negative values']

We want to check that the duration of all the periods are equal! We take the most common interval duration and see how many data points deviate from the mode. We want this to fail silently becuase it is probable that we will have some outlier fish with longer or shorter intervals.

In [14]:
# Make sure that all the time intervals are equal
def test_time_intervals(df, verbose):
    df_temp = df
    df_temp["length"] = df["end"] - df["start"]
    results = []
    
    # Find the mode of the column
    mode = df_temp["length"].mode()[0]
    
    outliers = df_temp.loc[df_temp["length"] != mode]
    
    results.append("The mode is {}".format(mode))
    results.append("{} rows deviate from the mode".format(outliers.shape[0]))
    
    # We only want to print out all the information for each individual fish 
    # if the user requests it to reduce the number of output rows.
    if verbose:
        for index, row in outliers.iterrows():
            results.append("The row at index {} has a duration of {}"
                           .format(index, round(row["length"], 2)))

    return results 

In [16]:
test_time_intervals(df, True)

['The mode is 60.0',
 '237 rows deviate from the mode',
 'The row at index 3229 has a duration of 76.6',
 'The row at index 3230 has a duration of 43.4',
 'The row at index 4118 has a duration of 20.2',
 'The row at index 7348 has a duration of 76.6',
 'The row at index 7349 has a duration of 43.4',
 'The row at index 8237 has a duration of 20.2',
 'The row at index 11467 has a duration of 76.6',
 'The row at index 11468 has a duration of 43.4',
 'The row at index 12356 has a duration of 20.2',
 'The row at index 15586 has a duration of 76.6',
 'The row at index 15587 has a duration of 43.4',
 'The row at index 16475 has a duration of 20.2',
 'The row at index 19705 has a duration of 76.6',
 'The row at index 19706 has a duration of 43.4',
 'The row at index 20594 has a duration of 20.2',
 'The row at index 23824 has a duration of 76.6',
 'The row at index 23825 has a duration of 43.4',
 'The row at index 24713 has a duration of 22.0',
 'The row at index 27943 has a duration of 76.6',


As you can see, a lot of fish have a duration different than the mode of 60 seconds. This would be important if we were to do data analysis on the fish's activity.  

The middur column is what we use for computing activity. We will first check the activity column middur. Obviously, fish cannot be spend more seconds active than the total duration of the period.

We see that there are 3 discrepancies per fish. Based on the trend of index numbers, we can assume that the start and the end of each session were of different durations than the bulk of the experimental data. We also see that indicies 40300 and 40301 have unique durations of 65.5 and 54.5. We hypothesize that that this is due to one period accidentally going long and the next period being cut short to compensate. 

Obviously, we would need to talk to the expirementer for accurate explanations, but data validation is helful in helping us know what questions to ask.

In [134]:
def test_middur(df):
    df_temp = df
    df_temp["length"] = df["end"] - df["start"]
    
    outliers = df_temp[df["middur"] > df_temp["length"]].shape[0]
    
    if outliers != 0:
        return "Something is wrong! In {} periods, fish ".format(outliers) + \
                "were active longer than the total duration"
    
    return "Number of seconds active is less than total period duration for all trials"

In [135]:
test_middur(df)

'Number of seconds active is less than total period duration for all trials'

We now know that are activity values are within the realm of possibility! 

Let's make sure that all the fish have the same number of data points. Once again, we want this to fail silently becuase it is probable that we will have some outlier fish with more or less datapoints.

In [245]:
def test_check_num_datapoints(df):
    NUM_POINTS_PER_FISH = 4119 # We know each fish should have 4119 data points
    
    fish_set = set(df["location"].values)
    results = []
    
    # Let's find out how many data points each fish has
    for fish in fish_set:
        fish_count = df[df["location"] == fish].shape[0]
        if fish_count != NUM_POINTS_PER_FISH:
            results.append("Fish number {} has {} points instead of {}" \
                           .format(fish, fish_count, NUM_POINTS_PER_FISH))
    if len(results) == 0:
        results.append("All the fish have {} datapoints! No need to fret!" \
                       .format(NUM_POINTS_PER_FISH))
    return results

In [246]:
test_check_num_datapoints(df)

['All the fish have 4119 datapoints! No need to fret!']

Now that we know that all the fish have the same number of data points, we can check that only the first entry has start reason 'Beginning of session' and only the last entry has end reason "End of session"

In [286]:
def test_start_end_reason(df):
    results = []
    if df["startreason"].iloc[0] != 'Beginning of session':
        results.append("Missing the beginning of session")
        
    if df["endreason"].iloc[df.shape[0] - 1 ] != "End of session":
        results.append("Missing the end of session")
    
    counter = 0
    for index, row in df.iterrows():
        if index % 4118 != counter:
            if df["endreason"].iloc[index] != "End of period":
                results.append("Row at index {} has an invalid endreason of {}" \
                               .format(index, df["endreason"].iloc[index] ))
            counter += 1
        else:
            print ("index: {}, counter : {}".format(index, counter))

       
        if index % 4119 != 0:
            if df["startreason"].iloc[index] != 'End of period':
                results.append("Row at index {} has an invalid startreason of {}" \
                               .format(index, df["startreason"].iloc[index]))
        
    # results.append("All the start and end reasons look ok!")
    return results

In [288]:
test_start_end_reason(df)

index: 0, counter : 0


['Row at index 4118 has an invalid endreason of End of session',
 'Row at index 8237 has an invalid endreason of End of session',
 'Row at index 12356 has an invalid endreason of End of session',
 'Row at index 16475 has an invalid endreason of End of session',
 'Row at index 20594 has an invalid endreason of End of session',
 'Row at index 24713 has an invalid endreason of End of session',
 'Row at index 28832 has an invalid endreason of End of session',
 'Row at index 32951 has an invalid endreason of End of session',
 'Row at index 37070 has an invalid endreason of End of session',
 'Row at index 41189 has an invalid endreason of End of session',
 'Row at index 45308 has an invalid endreason of End of session',
 'Row at index 49427 has an invalid endreason of End of session',
 'Row at index 53546 has an invalid endreason of End of session',
 'Row at index 57665 has an invalid endreason of End of session',
 'Row at index 61784 has an invalid endreason of End of session',
 'Row at ind

In [265]:
df[df["endreason"] == "End of session"]

,location,animal,user,sn,an,datatype,start,end,startreason,endreason,...,fredur,midct,middur,burct,burdur,stdate,sttime,genotype,test,length
4118,1,z001,ZEBRALAB02\zebralab_user,1,0,quant,247080.0,247100.2,End of period,End of session,...,20.2,0,0.0,0,0.0,20/07/2015,11:07:59,wt,-1,20.2
8237,2,z002,ZEBRALAB02\zebralab_user,1,0,quant,247080.0,247100.2,End of period,End of session,...,16.7,21,3.5,0,0.0,20/07/2015,11:07:59,mut,-2,20.2
12356,3,z003,ZEBRALAB02\zebralab_user,1,0,quant,247080.0,247100.2,End of period,End of session,...,18.9,7,1.3,0,0.0,20/07/2015,11:07:59,wt,-3,20.2
16475,4,z004,ZEBRALAB02\zebralab_user,1,0,quant,247080.0,247100.2,End of period,End of session,...,16.5,15,3.7,0,0.0,20/07/2015,11:07:59,mut,-4,20.2
20594,5,z005,ZEBRALAB02\zebralab_user,1,0,quant,247080.0,247100.2,End of period,End of session,...,18.4,9,1.8,0,0.0,20/07/2015,11:07:59,het,-5,20.2
24713,6,z006,ZEBRALAB02\zebralab_user,1,0,quant,247080.0,247102.0,End of period,End of session,...,20.7,9,1.2,1,0.1,20/07/2015,11:07:59,het,-6,22.0
28832,7,z007,ZEBRALAB02\zebralab_user,1,0,quant,247080.0,247102.1,End of period,End of session,...,22.0,1,0.1,0,0.0,20/07/2015,11:07:59,het,-7,22.1
32951,8,z008,ZEBRALAB02\zebralab_user,1,0,quant,247080.0,247102.1,End of period,End of session,...,19.9,14,2.2,0,0.0,20/07/2015,11:07:59,wt,-8,22.1
37070,9,z009,ZEBRALAB02\zebralab_user,1,0,quant,247080.0,247102.1,End of period,End of session,...,22.1,0,0.0,0,0.0,20/07/2015,11:07:59,het,-9,22.1
41189,10,z010,ZEBRALAB02\zebralab_user,1,0,quant,247080.0,247102.1,End of period,End of session,...,22.1,0,0.0,0,0.0,20/07/2015,11:07:59,het,-10,22.1


Enforce that fish 1-96 are here

Dates and times need to be in a specific format

Fish have equal # data points 

Numbers need to be non negative

intervals 60 sec and cont. -> fail silently

sn and an are binary -> what do they mean! They are always the same

check datatype all quant

middur is a double bw 0-60


Get rid of the c's in location combine with the other data frame and get rid of the NaNs

start and end excpet for the exception (max and min)

-----------
check for what the time interval is and then remove all the extraneous things. Only then can you really know what activity values to get rid of.

For question 2:
- calculate freq with fancy max function

- fit distribution to frequncy data 

Questions:
    
1. When we have a row duration of more that 60 seconds, should we throw it out?
2. So we are missing fish, good to know...
3. Is test_start_end_reason overkill?